# Preprocesamiento subtítulos

Este Notebook se encarga de la limpieza y preprocesamiento inicial de los subtítulos. Partiendo del texto del subtítulo de cada película, se siguen estos pasos:

1. [Quitar subtítulos que por error no son en inglés](#lang)
2. [Quedarse con archivos .srt](#srt) (son el 99%)
3. [Subtítulo a líneas](#subt_to_line)
4. [Lematizar](#lemas)
5. [Filtrar lemas y líneas](#filtro)
    
    
Input:
- Datasets con el texto del subtítulo de cada película: master_subt_content.pkl, con 3 variables centrales:
    - tconst (string): identificador película
    - s (string): texto del subtítulo
    - subt (string): nombre del archivo de subtítulo

Outputs:
- Archivos con el vocabulario único (vocab, stoi)
- Lemas limpios por película y por línea
    - Por película: master_subt_content_cleaned_lite.pkl, aquellas películas con in_cleaned == 1
    - Por línea: en archivos separados en una carpeta llamada linesdf_lemmas_filtered


In [ ]:
# Importamos librerías 

## Módulos generales
from libraries import *

## Módulos con funciones creadas para este trabajo
## (requieren de haber importado previamente los módulos generales)
from limpieza_subt import *


In [ ]:
# Completar con directorios 
gitwd =  ""
datawd = ""

## Importar datos
Archivo con el texto de los subtítulos para cada película: master_subt_content.pkl

In [ ]:
master_subt_content = pd.read_pickle(datawd + "/master_subt_content.pkl")  
print(master_subt_content.shape) 

In [ ]:
master_subt_content = master_subt_content[["tconst", "subt", "s"]]
master_subt_content.head(3)

<a id='lang'></a>
## Quitar subtítulos que por error no son en inglés

In [ ]:
lang = [langdetect(x) for x in tqdm(master_subt_content.s)]
lang = pd.DataFrame(lang)
lang.columns = ["lang", "prob_lang"]

In [ ]:
# ver qué idiomas tenemos
lang.lang.value_counts()

In [ ]:
# Unimos al dataset de subtitulos para ver algunos ejemplos
master_subt_content = pd.concat([master_subt_content, lang], axis = 1)
master_subt_content.s[master_subt_content.lang == "es"].reset_index(drop = True)[0][:100]

In [ ]:
# Nos quedamos con las películas que son en inglés con mayor probabilidad
master_subt_content = master_subt_content[master_subt_content.lang == "en"].reset_index(drop = True)
master_subt_content.lang.value_counts()

In [ ]:
# Analizamos los casos de baja probabilidad asignada al inglés: estan OK, mezclan idiomas o tienen algunas palabras raras, pero las sacaremos más adelante en la limpieza
master_subt_content.prob_lang.describe()

In [ ]:
master_subt_content.prob_lang[master_subt_content.prob_lang < 0.7].value_counts()

In [ ]:
master_subt_content.s[master_subt_content.prob_lang < 0.5].reset_index(drop = True)[0][:1000]

<a id='srt'></a>
## Agregar el tipo de archivo
Por el momento nos quedaremos solamente con los archivos .srt, a futuro extender las funciones de limpieza a otro tipo de archivos

In [ ]:
# add subtitles file type
master_subt_content["file_type"] = [fname[-4:] for fname in master_subt_content.subt]
print(master_subt_content.file_type.value_counts())

# for now keep just srt type of files, then extend cleaning functions to the other types [TBC]
print(f"Porcentaje de archivos tipo .srt: {round(np.mean([master_subt_content.file_type == '.srt']) * 100)} %")

master_subt_content = master_subt_content[master_subt_content.file_type == '.srt'].reset_index(drop = True)
print(master_subt_content.shape)

## Guardar versión hasta aquí

In [ ]:
master_subt_content[["tconst", "s"]].to_pickle(datawd + "/master_subt_content_clean.pkl")

<a id='subt_to_line'></a>
## Subtítulo a líneas

Obtener una base de datos al nivel de línea con marcas de tiempo.

Además: unidecode, lower case.

In [ ]:
n_films = master_subt_content.shape[0]

# Crear un dataset a nivel de línea llamado linesdf
linesdf = pd.DataFrame()

for i in tqdm(range(n_films)):
    
    cleanlines = subtitle_to_lines(master_subt_content.s[i])
    tconst =  master_subt_content.tconst[i]
    if len(cleanlines) > 0 :
        df = pd.DataFrame(cleanlines).reset_index() # index is the order of the line in the film
        df.columns = ['line_number', 'time', 'line']
        df['tconst'] = tconst
        linesdf = pd.concat([linesdf, df])
    
    # guardar cada 1000 películas para no tener un dataset enorme
    if i in np.arange(1000, n_films, 1000):
        linesdf.reset_index(drop = True, inplace = True) # reset general index
        linesdf.to_pickle(datawd + "/linesdfs/linesdf_master_batch" + str(i) + ".pkl")   
        linesdf = pd.DataFrame() # reset to empty dataset at the line level with cleaned lines
        
        
linesdf.to_pickle(datawd + "/linesdfs/linesdf_master_batch" + str(i) + ".pkl")   
del(linesdf)

In [ ]:
# Concatenar los datasets pequeños y guardarlos en un sólo dataset
linesdfs = os.listdir(datawd + "/linesdfs")
print(linesdfs)
linesdfs =[datawd + "/linesdfs/" + i for i in linesdfs]
linesdf = pd.DataFrame()
for i in tqdm(range(len(linesdfs))):
    df = pd.read_pickle(linesdfs[i])
    linesdf = pd.concat([linesdf, df])
    
linesdf.reset_index(drop = True, inplace = True)
linesdf.to_pickle(datawd + "/linesdf_master.pkl")  

<a id='lemas'></a>
## Lematizar

Usando un modelo de Spacy, tokenizamos, quitamos Stopwords, lematizamos cada línea del subtítulo

In [ ]:
linesdf = pd.read_pickle(datawd + "/linesdf_master.pkl")  

In [ ]:
print(linesdf.shape)
seed(9)
linesdf.sample(5)

In [ ]:
# Cargar Spacy model ycustomizar configuración
nlp = spacy.load("en_core_web_sm", 
                 exclude=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]) 

nlp.add_pipe("lemmatizer", config={"mode": "lookup"}).initialize() # lematizador usando look-up table

# agregar stopwords customizadas
nlp.Defaults.stop_words |= {'yeah', 'okay', 'yes', 'right', 'like', 'sure', 
                            'hey', 'hi', 'hello', 
                            'thing', 
                            'oh', 'huh', 'na', 'uh', 'ha', 'whoa', 
                            'ah', 'hmm', 'beep', 'uh', 'ah', 'wow', 
                            'way', 'um', 'ya', 'woaah', 'aha', 'ahem', 
                            'ahh', 'argh', 'aww', 'aw', 'bah', 'boo', 
                            'hoo', 'brr', 'duh', 'eek', 'eep', 'eh', 
                            'eww', 'fuff', 'gah', 'gee', 'grr', 'humph', 
                            'hah', 'haha', 'huh', 'hurrah', 'ick',
                            'meh', 'mhm', 'mm', 'muahaha', 'mwah', 
                            'nah', 'nuh', 'uh', 'ooh', 'la', 'oomph', 'oops',
                            'ouch', 'oww', 'oy', 'pew', 'pff', 'phew', 'psst', 'sheesh', 
                            'shh', 'shoo', 'tsk', 'umm', 'waah', 'wee', 'yahoo', 'yay',
                            'yee', 'haw', 'yikes', 'yoo', 'hoo', 'yuh', 'uh', 'yuck', 'zing'}

# customizar tokenización
suffixes = nlp.Defaults.suffixes + [r'''-''',]
suffix_regex = spacy.util.compile_suffix_regex(suffixes)
nlp.tokenizer.suffix_search = suffix_regex.search

prefixes = nlp.Defaults.prefixes + [r'''-''',]
prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [ ]:
# Ejemplo de lematización por línea
example = "--hey these (boys) and girls are dancing!!  !!!!!   !!!  "
doc = nlp(example)
lemmas = [t.lemma_ for t in doc if (not t.is_stop) & (not t.is_punct) & (not t.is_space) & (t.is_ascii) ]
lemmas

In [ ]:
# Lematizar todas las líneas
counter = Counter() # ir contando la cantidad de apariciones de cada lema

# de a subconjuntos del 4% de todos los datos (el proceso total lleva unas 7 horas)
n = linesdf.shape[0]
print("Total filas:", n)
step = int(np.floor(n/25))    
indexes = np.arange(0 , n-step , step).tolist() +  [n]
perc = 0

for i in range(len(indexes)-1):
    
    perc += 4
    
    # Quedarse con el 4% of dataset
    linesdf_lemmas = linesdf.iloc[ indexes[i]:  indexes[i+1], :].reset_index(drop = True)
    
    # Guardar columna con lemmas
    linesdf_lemmas["lemmas"] = np.nan
    row_count = 0
    
    # Spacy model
    docs = nlp.pipe(linesdf_lemmas.line, n_process=8, batch_size=2000) # procesamiento en paralelo
    
    for doc in tqdm(docs):
    
        # Tokenizar, quitar stopwords, quitar puntuación, lematizar
        lemmas = [t.lemma_ for t in doc if (not t.is_stop) & (not t.is_punct) & (not t.is_space) & (t.is_ascii)]
        
        # Guardar lemas concatenados en una columna
        linesdf_lemmas.loc[row_count,"lemmas"] = " ".join(lemmas)
        
        row_count += 1
        
        # Actualizar counter
        for lemma in lemmas:
            counter[lemma] += 1 # cuenta cuántas veces aparece cada token
    
    # Guardar
    linesdf_lemmas.to_pickle(datawd + "/linesdf_lemmas/linesdf_lemmas" + str(indexes[i+1])+ ".pkl")  
    with open(datawd + "/counter_lemmas.pkl", 'wb') as outputfile: 
        pickle.dump(counter, outputfile)
        
    print( str(perc) , "% completed")
    


In [ ]:
# Concatenar todos los datasets creados en el paso anterior para obtener un único dataset con los lemas
files = os.listdir(datawd + "/linesdf_lemmas")
linesdf_lemmas = pd.DataFrame()
for f in tqdm(files):
    df = pd.read_pickle(datawd + "/linesdf_lemmas/" + f)
    linesdf_lemmas = pd.concat([linesdf_lemmas, df]).reset_index(drop = True)
    
print(linesdf.shape[0])
print(linesdf_lemmas.shape[0])

# liberar memoria :)
del(linesdf) 

In [ ]:
a = linesdf_lemmas.shape[0] 

# quitar líneas que quedaron vacías o son sólo espacios (i.e. sólo tenían stopwords o puntuación)
linesdf_lemmas = linesdf_lemmas[~linesdf_lemmas.lemmas.str.isspace()].reset_index(drop = True)
linesdf_lemmas = linesdf_lemmas[linesdf_lemmas.lemmas != ""].reset_index(drop = True)

# Ver cuántas quitamos
print((linesdf_lemmas.shape[0] - a) / 1e6) 

In [ ]:
# GUARDAR
linesdf_lemmas = linesdf_lemmas.reset_index(drop = True)
linesdf_lemmas.to_pickle(datawd + "/linesdf_lemmas.pkl")

<a id='filtro'></a>

## Filtrar lemas y líneas
 1. [Lemas muy infrecuentes o muy comunes](#filtro-lema-freq-infreq)
 2. [Lemas OOV](#filtro-lema-oov)
 3. [Líneas y películas outliers en cantidad de lemas](#filtro-lema-outliers) 
 4. [Lemas en demasiadas o muy pocas películas](#filtro-lema-idf)
 5. [Vocabulario final y guardado de datasets](#filtro-final)

In [ ]:
# Importar datos
with open(datawd + "/counter_lemmas.pkl", 'rb') as inputfile: 
    counter = pickle.load(inputfile)
linesdf_lemmas = pd.read_pickle(datawd + "/linesdf_lemmas.pkl")

<a id='filtro-lema-freq-infreq'></a>
### 1. Lemas muy infrecuentes o muy comunes

In [ ]:
print(len(counter))  # cant. lemas únicos: ~ 478_000
plt.hist(counter.values(), bins = 30)
plt.yscale('log')
plt.show()

In [ ]:
sorted(counter.items(), key=lambda x: x[1])[:10]

In [ ]:
sorted(counter.items(), key=lambda x: x[1], reverse=True)[:30]

In [ ]:
# Quitar palabras muy poco frecuentes
for m in np.arange(0,51,10):
    min_frequency = m  # Umbral de frecuencia mínima a usar
    filtered_counter = Counter({word: freq for word, freq in counter.items() if freq >= min_frequency})
    print("Min. freq.:", m ,
          "- Number of unique lemmas:", len(filtered_counter),
          "- Words to remove:" , len(counter) - len(filtered_counter))

In [ ]:
# Umbrales de mínima y máxima frecuencia
min_frequency = 50  
max_frequency = 335_000  

# Distribución de frecuencias quitando esos lemas (filtrar lemas en el counter)
filtered_counter = Counter({word: freq for word, freq in counter.items() if (freq <= max_frequency) & (freq >= min_frequency) })
print("Number of unique lemmas:", len(filtered_counter), 
      "- Number of lemmas deleted:", len(counter) - len(filtered_counter))

plt.hist(filtered_counter.values(), bins = 30)
plt.yscale('log')
plt.show()

<a id='filtro-lema-oov'></a>
### 2. Lemas OOV

In [ ]:
model = api.load("glove-wiki-gigaword-300")  # Glove vectors

In [ ]:
# Dataframe a partir del counter filtrado en el paso anterior
cdf = pd.DataFrame.from_dict(filtered_counter, orient='index').reset_index()
cdf.columns = ["lemma", "counts"]

# Obtener vectores para cada lema
cdf["WORD_VECTORS"] = [get_word_vector(word, model = model) for word in tqdm(cdf.lemma)]
df_transformed = cdf.WORD_VECTORS.apply(pd.Series) # a columnas
df_transformed.columns = [f'dim_{i+1}' for i in range(len(df_transformed.columns))] # agregar dim_ al comienzo de cada columna pertenceciente al vector
cdf = pd.concat([ cdf[["lemma","counts"]], df_transformed ], axis = 1)
del(df_transformed)

# Obtener lemas donde todas las dimensiones son 0 (no tienen representación vectorial)
dims = [col for col in cdf.columns if "dim_" in col]
cdf["dimsum"] = cdf[dims].sum(axis = 1)

del(model)

Acerca de las palabras que no aparecen en Glove:
- Palabras abreviadas pero comunes: goin' doin' nothin', que fueron eliminadas como stopwords en sus mayoria (bleedin lovin no)
- Palabras con errores de tipeo o símbolos raros (ej. nbsp, \1c&hb0f0f0, uso de i intercambiado con l, falta o repeticion de caracteres)
- Insultos: motherfucking, shithead, douchebag
- Nombres y palabras poco comunes, tal vez en otro idioma (y solo queremos trabajar con subtítulos en inglés. Aunque eventualmente podría ser bueno ver que se usa otro idioma como predictor de que una película hable de inmigración o no, una limitación actual del trabajo es sólo trabajar con el vocabulario en inglés.
- Algunas que sí sería bueno eventualmente corregir: lndian ltalian (con "l" en vez de "I") covid covid-19

Aunque se podría hacer una limpieza refinada, son muy pocas las palabras frecuentes que no son stopwords y parecen contener información útil, con lo cual usaremos Glove como un método de limpieza, filtrando las palabras OOV, y luego podremos usar el mismo modelo como representación vectorial. Una alternativa a probar a furueo sería usar fasttext.

Además, quitamos algunos otros símbolos que veo que quedaron como tokens: "+", "f", "na", "=" , ">", "<". Debería mejorarse la limpieza previa para que también se quiten dichos símbolos.


In [ ]:
cdf.counts[cdf.dimsum == 0].hist(bins = 50)
plt.yscale('log')
plt.show()

In [ ]:
# Lemas y símbolos a quitar
remove_others = {"+", "f", "na", "=" , ">", "<"}
rmv_w2v = set(cdf.lemma[cdf.dimsum == 0]).union(remove_others)
print(len(rmv_w2v))

# Quitarlos del dataset de lemas y de Counter
cdf = cdf[~ cdf.lemma.isin(rmv_w2v)].reset_index(drop = True)
filtered_counter2 = Counter({word: freq for word, freq in filtered_counter.items() if word not in rmv_w2v })

# Chequear que coinciden cant. lemas en Counter y en dataset
print(cdf.shape[0] == len(filtered_counter2))

# Cantidad de lemas quitados
print(len(filtered_counter2) - len(filtered_counter))

In [ ]:
# Quitar lemas del dataset a nivel de línea
tokenizer = nlp.tokenizer
filtered2 = set(filtered_counter2.keys())

def filtered_words(text):    
    tokens = [t.text for t in tokenizer(text)]
    filtered_words = [t for t in tokens if t in filtered2 ]
    return (filtered_words)

linesdf_lemmas["filtered_words2"] = [filtered_words(x) for x in tqdm(linesdf_lemmas.lemmas, position=0, leave=True)]

In [ ]:
# Contar cantidad de lemas en cada línea hasta el momento
linesdf_lemmas["n_words"] = [len(x) for x in tqdm(linesdf_lemmas.filtered_words2, position=0, leave=True)]

In [ ]:
# Quitar las líneas que quedan vacías con el nuevo filtro
print(np.sum(linesdf_lemmas.n_words == 0))
linesdf_lemmas = linesdf_lemmas[linesdf_lemmas.n_words > 0].reset_index(drop = True)

<a id='filtro-lema-outliers'></a>
### 3. Líneas y películas outliers en cantidad de lemas

#### (a) Lemas por línea (quitar líneas outlier)

In [ ]:
linesdf_lemmas.n_words.hist()
plt.yscale('log')
plt.show()

In [ ]:
# Tomar las 50 líneas con más cantidad de lemas por líneas como outliers
q = 1 - 50 / linesdf_lemmas.shape[0] # calcular cuantil
print(q)

k = linesdf_lemmas.n_words.quantile(q) # mínima cantidad de lemas
print(k)

In [ ]:
outliers = linesdf_lemmas[linesdf_lemmas.n_words > k].sort_values("n_words", ascending = False)
outliers.head()

In [ ]:
outliers.n_words.hist(bins = 25)
plt.yscale('log')
plt.show()

In [ ]:
# Primero, verificamos que las películas a las que pertenecen las líneas outliers estén OK en general
check = linesdf_lemmas[linesdf_lemmas.tconst.isin(outliers.tconst.unique())]
check.groupby("tconst").head(5)
# Sí, lo están. Simplemente corregiremos las líneas correspondientes

In [ ]:
# Agregar una variable identificadora por película-línea (concatenar tconst y line_number) para facilitar el filtrado manual
linesdf_lemmas["lineid"] = linesdf_lemmas.tconst +"_" + linesdf_lemmas.line_number.astype("str")
linesdf_lemmas.head()

In [ ]:
outliers["lineid"] = outliers.tconst + "_" +  outliers.line_number.astype("str")

Se revisó manualmente las líneas seleccionadas. Se decidió:

Remover líneas:
- Con más de 200 lemas o con 52, 48, 32, 31 o 27 lemas
- tt1606197_72, tt8742774_31, tt8742774_43, tt8742774_900, tt9490414_527
- Todas las líneas con outliers en las películas: tt0117619, tt1606197, tt3684484, tt5516328

Corregir manualmente una línea:
- tt1847541_561 debe contener únicamente "california"


In [ ]:
# Aplicamos las correcciones
remove_lines = set(outliers.lineid[(outliers.n_words > 200) | 
                (outliers.n_words.isin([52, 48, 32, 31, 27])) |
                (outliers.tconst.isin(["tt0117619", "tt1606197", "tt3684484", "tt5516328",
                                      "tt0086203"])) |
                (outliers.lineid.isin(["tt1606197_72", "tt8742774_31", "tt8742774_43", "tt8742774_900", "tt9490414_527", 
                                       "tt8742774_49", "tt8742774_18", "tt8742774_47"
                                      ]))])

linesdf_lemmas.loc[linesdf_lemmas.lineid == "tt1847541_561", "filtered_words2"] = ["california"]
linesdf_lemmas.loc[linesdf_lemmas.lineid == "tt1847541_561", "n_words"] = 1

linesdf_lemmas = linesdf_lemmas[~linesdf_lemmas.lineid.isin(remove_lines)].reset_index(drop = True)

In [ ]:
#  Verificamos que funcionó
pd.set_option('display.max_rows',100)
outliers = linesdf_lemmas[linesdf_lemmas.n_words > k].sort_values("lineid", ascending = False)
outliers # funcionó!

In [ ]:
# Nuevo histograma de cantidad de lemas por línea
linesdf_lemmas.n_words.hist(bins = 20)
plt.yscale('log')
plt.show()

#### (b) Lemas por película (quitar películas outlier)

In [ ]:
# Contar también cantidad de caracteres alfabéticos
def count_alpha(text):    
    alphas = np.sum([t.isalpha() for t in text])
    return (alphas)

linesdf_lemmas["n_alphas"] = [count_alpha(x) for x in tqdm(linesdf_lemmas.filtered_words2, position=0, leave=True)]

In [ ]:
# Largo por película
lengthbyfilm = linesdf_lemmas.groupby('tconst').agg({'line': 'count',
                                                     'n_words':'sum',
                                                     'n_alphas': ['sum', 'mean']})
lengthbyfilm.columns = ["n_words",  "n_lines", "n_alphas", "mean_alpha"]
lengthbyfilm.describe()

Tenemos algunas películas con 0 caracteres alfabéticos, quitarlas.

In [ ]:
filmslowalpha = set(lengthbyfilm.index[(lengthbyfilm.mean_alpha < 1.2)])
print(len(filmslowalpha))

In [ ]:
lengthbyfilm = lengthbyfilm[~lengthbyfilm.index.isin(filmslowalpha)]
lengthbyfilm.describe()

Chequear las pelis con pocas líneas

In [ ]:
shortl = set(lengthbyfilm.index[(lengthbyfilm.n_lines < 50)])
print(len(shortl))

In [ ]:
# las revisamos manualmente
#pd.set_option('display.max_rows',400)
#linesdf_lemmas[linesdf_lemmas.tconst.isin(shortl)]

In [ ]:
lengthbyfilm = lengthbyfilm[~lengthbyfilm.index.isin(shortl)]
lengthbyfilm.describe()

In [ ]:
lengthbyfilm.hist(figsize = (20,5), bins = 50 )
plt.show()

Quitamos ambos conjuntos

In [ ]:
films_to_remove = filmslowalpha.union(shortl)
print(len(films_to_remove))

In [ ]:
linesdf_lemmas = linesdf_lemmas[~linesdf_lemmas.tconst.isin(films_to_remove )].reset_index(drop = True)
linesdf_lemmas.shape

<a id='filtro-lema-idf'></a>
### 4. Lemas en demasiadas o muy pocas películas
Basándonos en la IDF

In [ ]:
# Generar la matriz idf 

## partir del texto agrupado
df = linesdf_lemmas.groupby('tconst',as_index = False)["filtered_words2"].agg(list)
df["filtered_words2"] = [[item for sublist in l for item in sublist] for l in df.filtered_words2]
df["text_clean"] = [" ".join(x) for x in df.filtered_words2]

## calcular matrices usando sklearn
pipe = Pipeline([('count', CountVectorizer()),
                  ('tfid', TfidfTransformer())]).fit(df.text_clean)
vocab = pipe['count'].vocabulary_
ivocab = {v: k for k, v in vocab.items()}
idfs = pipe['tfid'].idf_
idfs     = pd.DataFrame(idfs)
    
# cambiar nombre de filas a los lemas en IDF
idfs = idfs.rename(index = ivocab)
idfs.columns = ["idf"]
idfs = idfs.sort_values("idf") # Aquellos con menor IDF son los lemas que aparecen en más películas

idfs.hist(bins = 50)
plt.show()

In [ ]:
common_words = set(idfs.head(25).index) # Son principalmente verbos
idfs.head(25)

In [ ]:
uncommon_words = idfs[idfs.idf >= 8] # mostly names
print(len(uncommon_words ))
#pd.set_option('display.max_rows',len(uncommon_words ))
#uncommon_words 

In [ ]:
# quitar ambos conjuntos
words_to_remove = set(uncommon_words.index).union(common_words)
len(words_to_remove)

In [ ]:
linesdf_lemmas.reset_index(drop = True, inplace = True)
linesdf_lemmas["filtered_words3"] = np.nan
linesdf_lemmas["filtered_words3"] = [[w for w in row if w not in words_to_remove] for row in tqdm(linesdf_lemmas.filtered_words2, position=0, leave=True)]

In [ ]:
linesdf_lemmas["n_words"] = [len(x) for x in tqdm(linesdf_lemmas.filtered_words3, position=0, leave=True)]

In [ ]:
linesdf_lemmas.shape

In [ ]:
linesdf_lemmas = linesdf_lemmas[linesdf_lemmas.n_words > 0].reset_index(drop = True)

<a id='filtro-final'></a>
### 5. Vocabulario final y guardado de datasets
Quitar los lemas infrecuentes luego del filtrado previo

In [ ]:
# Calcular la cantidad de apariciones
counter = Counter()
    for row in linesdf_lemmas.filtered_words3:
        for lemma in row:
            counter[lemma] += 1 

In [ ]:
# Crear vocabulario con lemas que aparecen al menos 50 veces
vocab = Vocab(
    counter,
    min_freq=50, 
    specials=["<unk>", "<pad>"],
    special_first=True
)

stoi = vocab.get_stoi()

UNK_IDX = stoi["<unk>"]
PAD_IDX = stoi["<pad>"]

len(vocab.get_stoi())

In [ ]:
def final_lemmas(example): # lemas que no son UNK
    final_lemmas  = [t  for t in example if stoi.get(t, UNK_IDX) != UNK_IDX] 
    return final_lemmas

def to_vocab(example): # token ids
    token_ids = [stoi.get(t, UNK_IDX)  for t in example] 
    return token_ids

In [ ]:
byfilm = pd.DataFrame() # create dataset at the film level

# Ir guardando en datasets a nivel de línea separados (si no es muy grande para leer en memoria)
n = len(set(linesdf_lemmas.tconst))
step = int(round(byfilm.shape[0] / 20))
indexes = np.arange(0 , n-step , step).tolist() +  [n]

for i in tqdm(range(len(indexes)-1), position=0, leave=True):
    
    # Quedarse con un subconjunto de películas
    films = set(byfilm.index[indexes[i]:  indexes[i+1]])
    df = linesdf_lemmas[linesdf_lemmas.tconst.isin(films)]
    
    
    try:
        
        df["final_lemmas"] = list(map(final_lemmas, df.filtered_words3)) # lemas que no son UNK
        df["token_ids"]    = list(map(to_vocab, df.final_lemmas)) # token ids
        df["n_tokens"]     =  list(map( lambda x: len(x), df.final_lemmas))

        df.drop(columns = ["lemmas", "filtered_words2", "n_words", "n_alphas", "filtered_words3"],
                   inplace = True)

        df = df[df.n_tokens > 0].reset_index(drop = True)

        df.to_pickle(datawd + "/linesdf_lemmas_filtered/linesdf_lemmas_filtered"+ str(indexes[i+1]) + ".pkl")

   
    
    # Dataset agrupando por película
    df = df.groupby('tconst',as_index = False).agg({"final_lemmas" : list,
                                                    "line": "count" ,
                                                   "token_ids": list,
                                                   "n_tokens": sum})
    df["final_lemmas"] = [[item for sublist in l for item in sublist] for l in df.final_lemmas]
    df["token_ids"] = [[item for sublist in l for item in sublist] for l in df.token_ids]
    
    byfilm = pd.concat([byfilm, df])

del(df)
byfilm = byfilm.rename(columns={'line': 'n_lines'})
byfilm.to_pickle(datawd + "/linesdf_lemmas_filtered/byfilm.pkl")

In [ ]:
print(byfilm.shape)
byfilm.head()

In [ ]:
# save vocab
with open(datawd + "/vocab.pkl", 'wb') as outputfile: # 'wb' means Write Binary. Instead, use 'rb' to read
    pickle.dump(vocab, outputfile)

# save stoi
with open(datawd + "/stoi.pkl", 'wb') as outputfile: # 'wb' means Write Binary. Instead, use 'rb' to read
    pickle.dump(stoi, outputfile)


In [ ]:
# agregamos las nuevas variables al dataset maestro de contenido de subtítulos
master = pd.read_pickle(datawd + "/master_subt_content.pkl")  
master["file_type"] = [fname[-4:] for fname in master.subt]
master = master.merge(byfilm,
                     on = "tconst",
                     how = "left",
                     indicator = True,
                     validate = "1:1")
del(byfilm)

master["in_cleaned"] = 0
master.loc[master._merge == "both", "in_cleaned"] = 1
master.drop(columns = ["_merge"], inplace = True)
master.head(2)

master.to_pickle(datawd + "/master_subt_content_cleaned.pkl")

In [ ]:
# Guardamos una versión más liviana
print(master.columns)

master[["tconst",
       "in_cleaned",
        "final_lemmas",
        "main",
        "before2000",
        "just_migra"   ]].to_pickle(datawd + "/master_subt_content_cleaned_lite.pkl")